## Additional exercise #1  (CSE 628) 

In [61]:
import nltk
from nltk.corpus import gutenberg
from nltk import bigrams, trigrams
import string
from decimal import *
import numpy as np
import random

#Setting decimal point precision
getcontext().prec = 6

In [62]:
#Importing data
new_data = []
data = gutenberg.sents('austen-sense.txt')
data = data[2:]

#Add sentence de-limiters, remove punctuations and switch to lower-case
for d in data:
    d = [''.join(c for c in s if c not in string.punctuation) for s in d]
    d = [s for s in d if s]
    d.insert(0,'<s>')
    d.append('</s>')
    d = [str(w.lower()) for w in d]
    new_data.append(d)
    
#The first 1000 sentences go into training set
train_set = new_data[0:1000]
#The second 1000 sentences go into test set
test_set = new_data[1000:2000]

train_words = [val for sublist in train_set for val in sublist]
num_words = len(train_words)

print "Size of training set: {} sentences and {} unique words".format(len(train_set), len(set(train_words)))
print "Size of test set: {} sentences and {} unqiue words".format(len(test_set), len(set([val for sublist in test_set for val in sublist])))

Size of training set: 1000 sentences and 2882 unique words
Size of test set: 1000 sentences and 2611 unqiue words


In [73]:
#Calculating bigrams
train_bigrams = list(bigrams(train_words))
#Calculating trigrams
train_trigrams = list(trigrams(train_words))

#Counting unigrams, bigrams and trigrams encountered in the training set
uni_count = dict([(item, train_words.count(item)) for item in sorted(set(train_words))])
bi_count = dict([(item, train_bigrams.count(item)) for item in sorted(set(train_bigrams))])
tri_count = dict([(item, train_trigrams.count(item)) for item in sorted(set(train_trigrams))])

print "Number of unigrams: {}".format(len(uni_count))
print "Number of bigrams: {}".format(len(bi_count))
print "Number of trigrams: {}".format(len(tri_count))

Number of unigrams: 2882
Number of bigrams: 14341
Number of trigrams: 21306


## Observation
We can observe that the number of unqiue N-grams increases on increasing the value of N. This is because, eg - repeating 3-word combinations (or trigrams) are rarer than repeating 2-word combinations (or bigrams), which in turn are rarer than repeating single words (or unigrams).

In [67]:
#Calculating MLE on test sentences using unigram model
test_uni_mle = []
for i in range(len(test_set)):
    uni_mle = 1
    for t in test_set[i][1:]:
        if t in uni_count.keys():
            uni_mle *= Decimal(uni_count[t])/Decimal(num_words)
        else:
            uni_mle = 0
            break
    test_uni_mle.append(uni_mle)

print "# of test sentences that get a non-zero probability according to unigram model: {}/{}".format(
    np.count_nonzero(test_uni_mle), len(test_set))

# of test sentences that get a non-zero probability according to unigram model: 388/1000


In [68]:
#Calculating MLE on test sentences using bigram model
test_bi_mle = []
for i in range(len(test_set)):
    bi_mle = 1
    test_bigrams = list(bigrams(test_set[i]))
    for j in range(len(test_bigrams)):
        if test_bigrams[j] in bi_count.keys():
            bi_mle *= Decimal(bi_count[test_bigrams[j]])/Decimal(uni_count[test_bigrams[j][0]])
        else:
            bi_mle = 0
            break
    test_bi_mle.append(bi_mle)

print "# of test sentences that get a non-zero probability according to bigram model: {}/{}".format(
    np.count_nonzero(test_bi_mle), len(test_set))

# of test sentences that get a non-zero probability according to bigram model: 29/1000


In [69]:
#Calculating MLE on test sentences using trigram model
test_tri_mle = []
for i in range(len(test_set)):
    tri_mle = 1
    test_trigrams = list(trigrams(test_set[i]))
    #if sentence only has less than 3 words, meaning no trigrams
    if len(test_trigrams) == 0:
        tri_mle = 0
        continue
    for j in range(len(test_trigrams)):
        if test_trigrams[j] in tri_count.keys():
            tri_mle *= Decimal(tri_count[test_trigrams[j]])/Decimal(bi_count[(test_trigrams[j][0],
                                                                              test_trigrams[j][1])])
        else:
            tri_mle = 0
            break
    test_tri_mle.append(tri_mle)

print "# of test sentences that get a non-zero probability according to trigram model: {}/{}".format(
    np.count_nonzero(test_tri_mle), len(test_set))

# of test sentences that get a non-zero probability according to trigram model: 16/1000


In [103]:
#Building multinomial probability distribution for uni, bi and trigrams
sampling_uni_count = uni_count.copy()
del sampling_uni_count['<s>']
del sampling_uni_count['</s>']
sampling_uni_list = list(sampling_uni_count.keys())
prob_uni = {k: Decimal(v) / Decimal(sum(sampling_uni_count.values())) for k, v in sampling_uni_count.iteritems()}
#prob_bi = {k: Decimal(v) / Decimal(uni_count[k[0]]) for k, v in bi_count.iteritems()}
#prob_tri = {k: Decimal(v) / Decimal(bi_count[k[0],k[1]]) for k, v in tri_count.iteritems()}

#Making up 5 non-sensical sentences from training set and calculating their MLE estimates
all_rands = []
print "Generated random sentences:"
for i in range(5):
    #Select sentence length at random
    sent_len = random.sample(range(5,10),1)
    rand_sent = []
    for j in range(sent_len[0]):
        sample = np.random.multinomial(1,prob_uni.values(),size=1)
        rand_sent.append(sampling_uni_list[np.nonzero(sample[0])[0][0]])
    rand_sent.insert(0,'<s>')
    rand_sent.append('</s>')
    all_rands.append(rand_sent)
    print ' '.join(rand_sent)

Generated random sentences:
<s> embarrassment sail meet affections 8 laugh </s>
<s> stone read rent eat aggrandizement enquiry preserver 8 deficiencies </s>
<s> appearance eat reflecting compass argument </s>
<s> stone power preserver aggrandizement gradually </s>
<s> already rival insipidity gradually mistake staircase </s>


In [104]:
rand_uni_mle = []
for i in range(len(all_rands)):
    uni_mle = 1
    for t in all_rands[i][1:]:
        if t in uni_count.keys():
            uni_mle *= Decimal(uni_count[t])/Decimal(num_words)
        else:
            uni_mle = 0
            break
    rand_uni_mle.append(uni_mle)

print "Unigram MLEs for the random sentences: \n1. {}\n2. {}\n3. {}\n4. {}\n5. {}\n".format(rand_uni_mle[0], 
                                                rand_uni_mle[1], rand_uni_mle[2], rand_uni_mle[3], rand_uni_mle[4])

rand_bi_mle = []
for i in range(len(all_rands)):
    bi_mle = 1
    rand_bigrams = list(bigrams(all_rands[i]))
    for j in range(len(rand_bigrams)):
        if rand_bigrams[j] in bi_count.keys():
            bi_mle *= Decimal(bi_count[rand_bigrams[j]])/Decimal(uni_count[rand_bigrams[j][0]])
        else:
            bi_mle = 0
            break
    rand_bi_mle.append(bi_mle)

print "Bigram MLEs for the random sentences: \n1. {}\n2. {}\n3. {}\n4. {}\n5. {}\n".format(rand_bi_mle[0], 
                                                rand_bi_mle[1], rand_bi_mle[2], rand_bi_mle[3], rand_bi_mle[4])

rand_tri_mle = []
for i in range(len(all_rands)):
    tri_mle = 1
    rand_trigrams = list(trigrams(all_rands[i]))
    #if sentence only has less than 3 words, meaning no trigrams
    if len(rand_trigrams) == 0:
        tri_mle = 0
        continue
    for j in range(len(rand_trigrams)):
        if rand_trigrams[j] in tri_count.keys():
            tri_mle *= Decimal(tri_count[rand_trigrams[j]])/Decimal(bi_count[(rand_trigrams[j][0],
                                                                              rand_trigrams[j][1])])
        else:
            tri_mle = 0
            break
    rand_tri_mle.append(tri_mle)

print "Trigram MLEs for the random sentences: \n1. {}\n2. {}\n3. {}\n4. {}\n5. {}\n".format(rand_tri_mle[0], 
                                            rand_tri_mle[1], rand_tri_mle[2], rand_tri_mle[3], rand_tri_mle[4])


Unigram MLEs for the random sentences: 
1. 1.51805E-27
2. 1.55200E-40
3. 1.85816E-23
4. 5.10996E-23
5. 3.79514E-27

Bigram MLEs for the random sentences: 
1. 0
2. 0
3. 0
4. 0
5. 0

Trigram MLEs for the random sentences: 
1. 0
2. 0
3. 0
4. 0
5. 0



## ADD - 1 SMOOTHING
### Unigram Model
As the words unseen in training set and occuring in the test set will result in 0 MLE, we take the following steps to prevent that - 
* Add a new token in the dictionary (formed from training set) called 'UNK', which will represent all the never seen before words in encountered in the test set.
* Add 1 to the counts of all the words in this modified dictionary.
* The new formula of MLE will now be: 
$$Pr_{MLE}(w_i) = \dfrac{count(w_i) + 1}{\Sigma_{w_j \in V}count{(w_j)} + V}$$

In [54]:
#Implementing add-1 smoothing with unigram model. V = #(train + test) words

#Adding 1 to the counts of existing entries
for t in uni_count:
    uni_count[t] = uni_count[t] + 1

#Adding UNK token
uni_count['UNK'] = 1

V = len(uni_count)

add1_uni_mle = []
for i in range(len(test_set)):
    uni_mle = 1
    for t in test_set[i][1:]:
        if t in uni_count.keys():
            uni_mle *= Decimal(uni_count[t])/Decimal(num_words+V)
        else:
            uni_mle *= Decimal(uni_count['UNK'])/Decimal(num_words+V)
            break
    add1_uni_mle.append(uni_mle)

print "# of test sentences that get a non-zero probability according to unigram model (add-1 smoothing): {}/{}".format(
    np.count_nonzero(add1_uni_mle), len(test_set))

# of test sentences that get a non-zero probability according to unigram model (add-1 smoothing): 1000/1000


## Bigram Model
* Similarly all the bigrams appearing in test set which didn't appear in the training set are represented by the 'UNK' token. Add 1 to the counts of all the tokens in our new vocabulary.
* The new formula for MLE will now be:
$$ Pr(w_j|w_i) = \dfrac{count(w_i,w_j) + 1}{count(w_i) + V}$$

In [55]:
#Implementing add-1 smoothing with bigram model.

all_test_bigrams = list(bigrams(test_words))

#Adding 1 to the counts of existing entries
for t in bi_count:
    bi_count[t] = bi_count[t] + 1

#Adding UNK token
bi_count['UNK'] = 1

add1_bi_mle = []
for i in range(len(test_set)):
    bi_mle = 1
    test_bigrams = list(bigrams(test_set[i]))
    for j in range(len(test_bigrams)):
        if test_bigrams[j] in bi_count.keys():
            bi_mle *= Decimal(bi_count[test_bigrams[j]])/Decimal(uni_count[test_bigrams[j][0]]+V)
        else:
            bi_mle *= Decimal(bi_count['UNK'])/Decimal(uni_count[test_bigrams[j][0]]+V)

    add1_bi_mle.append(bi_mle)

print "# of test sentences that get a non-zero probability according to bigram model (add-1 smoothing): {}/{}".format(
    np.count_nonzero(add1_bi_mle), len(test_set))

# of test sentences that get a non-zero probability according to bigram model (add-1 smoothing): 1000/1000
